In [33]:
import os
import time
import torch
import random
import numpy as np
import open3d as o3d
from SimAttention.dataloader import ModelNetDataSet

root = '/home/akira/下载/Pointnet2_PyTorch-master/byol_pcl/data/modelnet40_normal_resampled'
data = ModelNetDataSet(root)
DataLoader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True)

demo = 0
for data, cls in DataLoader:
    demo = data
    break
numpy_demo = demo.numpy()

The size of train data is 9843


In [2]:
def visualize_samples(d_sample):
    print("Points in downsample set: ", len(d_sample))
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(d_sample)
    color = [102.0 / 255.0 ,111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    o3d.visualization.draw_geometries([source])

In [37]:
visualize_samples(numpy_demo[0])

Points in downsample set:  10000


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [38]:
def index_points(points, idx):
    raw_size = idx.size()
    idx = idx.reshape(raw_size[0], -1)
    res = torch.gather(points, 1, idx[..., None].expand(-1, -1, points.size(-1)))
    return res.reshape(*raw_size, -1)

def b_FPS(xyz, npoint):
    device = xyz.device
    B, N, C = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
    distance = torch.ones(B, N).to(device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)
    batch_indices = torch.arange(B, dtype=torch.long).to(device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        distance = torch.min(distance, dist)
        farthest = torch.max(distance, -1)[1]
    return centroids, index_points(xyz, centroids)

def k_patch(x, k=2048, n_patch=8, n_points=1024):
    # 首先是用fps算法选择出来8个中心点，下面是中心点的索引
    patch_centers_index, _ = b_FPS(x, n_patch) # torch.Size([4, 8])
    center_point_xyz = index_points(x, patch_centers_index)
    print("patch_center_index shape: ", patch_centers_index.shape)
    # knn部分
    x = x.permute(0, 2, 1)
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
    print('pairwise_distance shape: ', pairwise_distance.shape) # torch.Size([4, 10000, 10000])
    # (batch_size, num_points, k) ---> bs, n_patch, k 
    idx = pairwise_distance.topk(k=k, dim=-1)[1][:,patch_centers_index[1],:]
    # idx = pairwise_distance.topk(k=k, dim=-1)[1]
    print('idx shape: ', idx.shape) # torch.Size([4, 8, 2048])
    idx = idx.permute(0, 2, 1)  # 4, 2048, 8
    # new patch points
    new_patch = torch.zeros([8, 4, 1024, 3]).to(device)
    for i in range(n_patch):
        patch_idx = idx[:,:,i].reshape(x.shape[0], -1)
        _, patch_points = b_FPS(index_points(x.permute(0, 2, 1), patch_idx), n_points)
        new_patch[i] = patch_points
    new_patch = new_patch.permute(1, 0, 2, 3)
#     print('new_patch shape: ', new_patch.shape)
    return new_patch, center_point_xyz

In [39]:
k_result, center_xyz = k_patch(demo)


patch_center_index shape:  torch.Size([4, 8])
pairwise_distance shape:  torch.Size([4, 10000, 10000])
idx shape:  torch.Size([4, 8, 2048])


In [40]:
center_xyz

tensor([[[ 0.3680, -0.1663, -0.1538],
         [-0.3768,  0.7097,  0.5074],
         [-0.4264, -0.6518,  0.3977],
         [-0.4033, -0.6554, -0.6309],
         [-0.4492,  0.1731, -0.2961],
         [ 0.3751,  0.4394,  0.3850],
         [ 0.4082, -0.6544,  0.4020],
         [ 0.4021, -0.6556, -0.6317]],

        [[ 0.4742, -0.0249, -0.7958],
         [-0.5871,  0.2916,  0.5723],
         [ 0.5709, -0.2041,  0.5687],
         [-0.5891, -0.0745, -0.4885],
         [ 0.1046,  0.0790, -0.0578],
         [-0.1146, -0.2038,  0.5770],
         [-0.5383, -0.1435,  0.1235],
         [ 0.2405,  0.2969,  0.5616]],

        [[ 0.3434, -0.1399, -0.4659],
         [-0.1261,  0.0971,  0.7936],
         [-0.7648, -0.0226, -0.2156],
         [ 0.3415, -0.1806,  0.2926],
         [-0.2652,  0.1325, -0.6299],
         [-0.1348,  0.0221, -0.0358],
         [ 0.3412, -0.1034,  0.7938],
         [ 0.3413,  0.1368, -0.0697]],

        [[ 0.4400,  0.0247, -0.2977],
         [-0.6516, -0.6462,  0.3972],
      

In [57]:
center = center_xyz[0].cpu().numpy()
visualize_samples(center)

Points in downsample set:  8


In [56]:
k = k_result[0][7]
k = k.cpu().numpy()
visualize_samples(k)

Points in downsample set:  1024


In [13]:
k.shape

(1024, 3)

In [14]:
k

array([[ 0.5984228 , -0.00322462, -0.3226279 ],
       [-0.6857589 ,  0.11277416, -0.07336222],
       [-0.05282892, -0.1530572 ,  0.24673198],
       ...,
       [ 0.5984228 , -0.00322462, -0.3226279 ],
       [ 0.5984228 , -0.00322462, -0.3226279 ],
       [ 0.5984228 , -0.00322462, -0.3226279 ]], dtype=float32)